In [2]:
import numpy as np
import pandas as pd
import torch

from pathlib import Path
import sys
sys.path.insert(0, str(Path.cwd().resolve().parents[0] / '2_Propensities'))

import MF_class as MF

np.random.seed(42)
if np.random.choice(np.arange(1000)) != 102:
    raise ValueError("Random seed is not set correctly.")

```
                                USERS                             
         ┌───────────────────────────────────────────────────────┐
         │                          │                            │
         │                          │                            │
         │                          │                            │
         │                          │                            │
ITEMS    │                          │                            │
         │                          │                            │
         │                          │                            │
         ├──────────────────────────┼────────────────────────────┤
         │                          │████████████████████████████│
         │                          │████████████████████████████│
         └───────────────────────────────────────────────────────┘
```

# 1. Load Data

In [3]:
base_artifacts = Path.cwd().resolve().parents[1] / 'CausalI2I_artifacts'
data_path = base_artifacts / 'Datasets' / 'Sequels'

train = pd.read_csv(data_path / 'train.csv')
test = pd.read_csv(data_path / 'test.csv')

n_users = train['user_id'].nunique()
n_items = train['item_id'].nunique()
print(f'Number of users: {n_users}, Number of items: {n_items}')

Number of users: 7801, Number of items: 6384


# 2. Train Model

In [10]:
model = MF.MatrixFactorizationTorch(n_users, n_items, n_factors=25)
model.fit(
    train_data=train.values,
    val_data=test.values,
    lr=5e-4, 
    wd=1e-7,
    pos_weight=1,
    batch_size=2**15,
    n_epochs=40,
    device=torch.device('cuda:0'), 
    use_amp=True)

Epoch  ||- - - - - - - - Train - - - - - - - -||- - - - - - Validation - - - - - - - || Epoch's | COS θ | Time     
Number || BCE    | BCE-POS | BCE-NEG | MPR    || BCE    | BCE-POS | BCE-NEG | MPR    || Change  |       | Elapsed  
=======||========|=========|=========|========||========|=========|=========|========||=========|=======|==========
   1   || 0.0783 |  3.1417 |  0.0292 | 0.7644 || 0.1340 |  2.6369 |  0.0492 | 0.7711 || 194.12  | None  | 00:07.77
   2   || 0.0713 |  3.4861 |  0.0167 | 0.7823 || 0.1259 |  2.8651 |  0.0330 | 0.7788 ||  33.70  | 0.839 | 00:15.60
   3   || 0.0709 |  3.4900 |  0.0161 | 0.7850 || 0.1253 |  2.8674 |  0.0324 | 0.7799 ||  10.29  | 0.708 | 00:23.48
   4   || 0.0705 |  3.4703 |  0.0160 | 0.7888 || 0.1247 |  2.8542 |  0.0322 | 0.7827 ||  12.72  | 0.633 | 00:31.22
   5   || 0.0693 |  3.4053 |  0.0159 | 0.7996 || 0.1229 |  2.8017 |  0.0321 | 0.7921 ||  20.98  | 0.911 | 00:39.10
   6   || 0.0676 |  3.3033 |  0.0157 | 0.8165 || 0.1202 |  2.7266 |  0.0319 |

### Save Model

In [ ]:
model.save(path=base_artifacts / 'Propensity_Models' / 'MF25_sequels.pt', note=None)

### Load Model

In [5]:
loaded_model = MF.MatrixFactorizationTorch(n_users, n_items, n_factors=25)
loaded_model.load(path=base_artifacts / 'Propensity_Models' / 'MF25_sequels.pt')

Loaded model summary:
Model:                      MatrixFactorizationTorch
Number of users:            7801
Number of items:            6384
Number of factors:          25
Learning rate:              0.0005
Weight decay:               1e-07
Positive weight:            1
Batch size:                 32768
Number of epochs:           40
Device:                     cuda:0
Use AMP:                    True
Timestamp:                  2026-01-23 15:04:37
